# import lib and function

In [ ]:
import importlib
import Utils.path_file_loader as pfl
importlib.reload(pfl)
print(dir(pfl))


['Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'os', 'target_path', 'test_reload']


<module 'Utils.path_file_loader' from 'd:\\KMITL\\year_4\\Intro_to_DL_and_ML\\Project\\Telco-Churn-Prediction-Model-Intro-to-ML-and-DL-Project-\\Utils\\path_file_loader.py'>

In [2]:
import kagglehub
import pandas as pd
import os
import numpy as np
from Utils.path_file_loader import target_path
from pathlib import Path
from pyspark.sql import DataFrame

c:\Users\korph\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# get dataframe

In [3]:
df = pd.read_csv(target_path("DataBase\Cleaned_data.csv"))
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# inspect information

In [61]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [77]:
churn_percent = df["Churn"].value_counts()
print(churn_percent)

Churn
No     5174
Yes    1869
Name: count, dtype: int64


In [62]:
churn_percent = df["Churn"].value_counts(normalize=True) * 100
print(churn_percent)

Churn
No     73.463013
Yes    26.536987
Name: proportion, dtype: float64


In [68]:
inspect = df.sort_values(by="TotalCharges", ascending=False)
inspect = inspect[['tenure','PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']]

inspect["tenure_x_MonthlyCharges"] = (inspect["tenure"] * inspect["MonthlyCharges"]).astype(int)
inspect['TotalCharges'] = (inspect['TotalCharges']).astype(float)
inspect['diff'] = (inspect['tenure_x_MonthlyCharges'] - inspect['TotalCharges']) / inspect['tenure']
inspect.head()


,tenure,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_x_MonthlyCharges,diff
4610,72,Bank transfer (automatic),117.80,8684.80,Yes,8481,-2.830556
4586,72,Bank transfer (automatic),118.75,8672.45,No,8550,-1.700694
6768,72,Credit card (automatic),117.50,8670.10,No,8460,-2.918056
5347,72,Credit card (automatic),116.95,8594.40,No,8420,-2.422222
2187,71,Electronic check,116.25,8564.75,No,8253,-4.390845


In [ ]:
top_lost = inspect[df['Churn'] == 'Yes']



top_lost.head()

C:\Users\korph\AppData\Local\Temp\ipykernel_6412\1707055771.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  top_lost = inspect[df['Churn'] == 'Yes']


,tenure,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_x_MonthlyCharges,diff
4610,72,Bank transfer (automatic),117.80,8684.80,Yes,8481,-2.830556
1306,70,Credit card (automatic),115.55,8127.60,Yes,8088,-0.565714
3890,72,Electronic check,109.25,8109.80,Yes,7866,-3.386111
6537,70,Credit card (automatic),115.65,7968.85,Yes,8095,1.802143
5581,68,Electronic check,113.15,7856.00,Yes,7694,-2.382353


# analysis business impact

In [79]:
churn_df = df[df['Churn'] == 'Yes']
lost_per_month = churn_df['MonthlyCharges'].sum()
print(f"With {churn_percent['Yes']} churn user make lost per month : {lost_per_month} dollar, if can save at 30 percent , the value of model is {lost_per_month* 0.3} dollar")

With 1869 churn user make lost per month : 139130.85 dollar, if can save at 30 percent , the value of model is 41739.255 dollar
